# Convolutional Neural Network Structure optimization Using Genetic

**author:** Mohammad reza Abbasniya

**date:** July 2023

#### Import libraries and packages

In [1]:
import tensorflow as tf
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.utils.np_utils import to_categorical

#### prepare data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

train_images = x_train.astype('float32')/255
test_images = x_test.astype('float32')/255

train_labels=to_categorical(y_train)
test_labels=to_categorical(y_test)

val_images=train_images[:10000]
partial_images=train_images[10000:]

val_labels=train_labels[:10000]
partial_labels=train_labels[10000:]

print(f'''
Train:       X: {partial_images.shape}
             y: {partial_labels.shape}

Validation:  X: {val_images.shape}
             y: {val_labels.shape}

Test:        X: {test_images.shape}
             y: {test_labels.shape}
''')

170498071/170498071 [==============================] - 5s 0us/step

Train:       X: (40000, 32, 32, 3)
             y: (40000, 10)

Validation:  X: (10000, 32, 32, 3)
             y: (10000, 10)

Test:        X: (10000, 32, 32, 3)
             y: (10000, 10)



#### Run Genetic

In [3]:
from datetime import datetime
from genetic_cnn_optimizer import GeneticCNN_finder
map = {}

def get_CNN(chromo):
  model = models.Sequential()

  if chromo[0] == 3:
    model.add(layers.Conv2D(chromo[1], chromo[2], input_shape=(32,32,3), activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[3], chromo[4], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[5], chromo[6], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation="softmax"))

  elif chromo[0] == 5:
    model.add(layers.Conv2D(chromo[1], chromo[2], input_shape=(32,32,3), activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[3], chromo[4], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[5], chromo[6], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[7], chromo[8], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[9], chromo[10], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation="softmax"))

  elif chromo[0] == 7:
    model.add(layers.Conv2D(chromo[1], chromo[2], input_shape=(32,32,3), activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[3], chromo[4], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[5], chromo[6], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[7], chromo[8], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(chromo[9], chromo[10], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[11], chromo[12], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(chromo[13], chromo[14], activation = "relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation="softmax"))

  return model


def accuracy_tester(chromo):
    start_time = datetime.now()
    print(f'evaluating chromo: {chromo} ')

    if tuple(chromo) in map:
        accuracy = map[tuple(chromo)]
    else:
        model = get_CNN(chromo)
        # model.summary()

        es = EarlyStopping(monitor="val_accuracy", patience=5)
        op = optimizers.Adam(0.01)
        model.compile(loss="categorical_crossentropy", optimizer=op, metrics=["accuracy"])
        model.fit(partial_images, partial_labels,
                  validation_data=(val_images, val_labels),
                  epochs=12, callbacks=[es], verbose=0)

        loss, accuracy = model.evaluate(test_images, test_labels)
        map[tuple(chromo)] = accuracy

    time_elapsed = datetime.now() - start_time
    print(f'===> time-taken: [{time_elapsed}]     test-accuracy: [{accuracy * 100:.2f}%] \n')
    return accuracy


gcf = GeneticCNN_finder(
    max_gen=5,

    cross_prob=0.6,
    mutation_prob=0.3,
    max_population=6,
    survive_percent=0.4,

    random_state=0,
    accuracy_function=accuracy_tester
)

elit = gcf.run()

evaluating chromo: [5, 128, 3, 64, 7, 128, 5, 64, 5, 64, 7] 
313/313 [==============================] - 2s 6ms/step - loss: 0.7317 - accuracy: 0.7841
===> time-taken: [0:05:30.276736]     test-accuracy: [78.41%] 

evaluating chromo: [3, 256, 3, 64, 3, 16, 7] 
313/313 [==============================] - 2s 5ms/step - loss: 0.9951 - accuracy: 0.6942
===> time-taken: [0:03:26.728930]     test-accuracy: [69.42%] 

evaluating chromo: [5, 256, 7, 256, 3, 64, 3, 16, 7, 64, 5] 
313/313 [==============================] - 2s 7ms/step - loss: 0.9658 - accuracy: 0.7000
===> time-taken: [0:04:27.309403]     test-accuracy: [70.00%] 

evaluating chromo: [7, 16, 5, 128, 5, 256, 7, 32, 7, 128, 5, 256, 5, 16, 7] 
313/313 [==============================] - 3s 8ms/step - loss: 1.0368 - accuracy: 0.7333
===> time-taken: [0:06:27.828644]     test-accuracy: [73.33%] 

evaluating chromo: [3, 16, 7, 128, 7, 16, 7] 
313/313 [==============================] - 1s 4ms/step - loss: 0.8987 - accuracy: 0.7002
===> tim

In [4]:
print(f'''FINAL RESULT:
    chromosome  : {elit.chromo}
    accuracy    : {elit.accuracy*100:.2f}%
 \n\n''')

final_model = get_CNN(elit.chromo)
final_model.summary()

FINAL RESULT:
    chromosome  : [7, 128, 3, 64, 7, 128, 5, 64, 5, 64, 7, 256, 7, 64, 3]
    accuracy    : 78.82%
 


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_124 (Conv2D)         (None, 30, 30, 128)       3584      
                                                                 
 batch_normalization_124 (Ba  (None, 30, 30, 128)      512       
 tchNormalization)                                               
                                                                 
 conv2d_125 (Conv2D)         (None, 30, 30, 64)        401472    
                                                                 
 batch_normalization_125 (Ba  (None, 30, 30, 64)       256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 15, 15, 64)       0         
 g